In [1]:
import os
import functools
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
tf_records_filenames = [os.path.join('../data_patch', fn) for fn in os.listdir('../data_patch')]
tf_records_filenames

['../data_patch/val.tfrecords',
 '../data_patch/test.tfrecords',
 '../data_patch/train.tfrecords']

In [4]:
c = 0
for fn in tf_records_filenames:
    for record in tf.python_io.tf_record_iterator(fn):
        c += 1
    print(fn, c)

../data_patch/val.tfrecords 1380
../data_patch/test.tfrecords 4166
../data_patch/train.tfrecords 20262


In [5]:
from dataset_utils import ConvLSTMDataSet

In [6]:
dataset = ConvLSTMDataSet()

In [7]:
in_steps = 14
out_steps = 1
HEIGHT = 32
WIDTH = 32
DEPTH = 1


class ConvLSTMDataSet_1(object):

  def __init__(self, data_dir='../data', subset='train', use_distortion=True):
    self.data_dir = data_dir
    self.subset = subset
    self.use_distortion = use_distortion

  def get_filenames(self):
    if self.subset in ['train', 'val', 'test']:
      return [os.path.join(self.data_dir, self.subset + '.tfrecords')]
    else:
      raise ValueError('Invalid data subset "%s"' % self.subset)

  def parser(self, serialized_example):
    """Parses a single tf.Example into image and label tensors."""
    # Dimensions of the images in the CIFAR-10 dataset.
    # See http://www.cs.toronto.edu/~kriz/cifar.html for a description of the
    # input format.
    features = tf.parse_single_example(
        serialized_example,
        features={
            'inputs': tf.FixedLenFeature([in_steps * DEPTH * HEIGHT * WIDTH], tf.float32),
            'labels': tf.FixedLenFeature([out_steps * DEPTH * HEIGHT * WIDTH], tf.float32),
        })

    # Reshape from [in_steps * depth * height * width] to [in_steps, height, width, depth].
    inputs = tf.reshape(features['inputs'], [in_steps, HEIGHT, WIDTH, DEPTH])
    labels = tf.reshape(features['labels'], [out_steps, HEIGHT, WIDTH, DEPTH])

    return inputs, labels


  def make_batch(self, batch_size, epochs=2):
    """Read the images and labels from 'filenames'."""
    filenames = self.get_filenames()
    # Repeat infinitely.
    dataset = tf.data.TFRecordDataset(filenames).repeat(1)

    # Parse records.
    dataset = dataset.map(
        self.parser, num_parallel_calls=batch_size)

    # Potentially shuffle records.
    if self.subset == 'train':
      min_queue_examples = int(
          ConvLSTMDataSet.num_examples_per_epoch(self.subset) * 0.4)
      # Ensure that the capacity is sufficiently large to provide good random
      # shuffling.
      dataset = dataset.shuffle(buffer_size=min_queue_examples + 3 * batch_size)

    # Batch it up.
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    inputs_batch, labels_batch = iterator.get_next()

    return inputs_batch, labels_batch

  @staticmethod
  def num_examples_per_epoch(subset='train'):
    if subset == 'train':
      return 20096
    elif subset == 'val':
      return 1380
    elif subset == 'test':
      return 4146
    else:
      raise ValueError('Invalid data subset "%s"' % subset)

In [8]:
dataset = ConvLSTMDataSet_1(subset='val')

In [9]:
inputs_batch, labels_batch = dataset.make_batch(32)

In [10]:
print(inputs_batch[1])

Tensor("strided_slice:0", shape=(14, 32, 32, 1), dtype=float32)


In [12]:
def input_fn(data_dir, subset, batch_size,
             use_distortion_for_training=True):
    use_distortion = subset == 'train' and use_distortion_for_training
    dataset = ConvLSTMDataSet(data_dir, subset, use_distortion)
    return dataset.make_batch(batch_size)

In [13]:
data_dir = '../data_patch'
batch_size = 1

In [14]:
train_inpf = functools.partial(input_fn, data_dir, 'train', batch_size)
eval_inpf = functools.partial(input_fn, data_dir, 'val', batch_size)

In [19]:
inputs, labels = train_inpf()

In [20]:
a = set()

In [21]:
with tf.Session() as sess:
    for i in range(21000):
        inputs1, labels1 = sess.run([inputs, labels])
        a.add(inputs1.mean())
        #plt.figure()
        #plt.imshow(labels1.squeeze())

In [22]:
len(a)

14903

In [37]:
def parser(serialized_example):
    """Parses a single tf.Example into image and label tensors."""
    # Dimensions of the images in the CIFAR-10 dataset.
    # See http://www.cs.toronto.edu/~kriz/cifar.html for a description of the
    # input format.
    features = tf.parse_single_example(
        serialized_example,
        features={
            'inputs': tf.FixedLenFeature([in_steps * DEPTH * HEIGHT * WIDTH], tf.float32),
            'labels': tf.FixedLenFeature([out_steps * DEPTH * HEIGHT * WIDTH], tf.float32),
        })

    # Reshape from [in_steps * depth * height * width] to [in_steps, height, width, depth].
    inputs = tf.reshape(features['inputs'], [in_steps, HEIGHT, WIDTH, DEPTH])
    labels = tf.reshape(features['labels'], [out_steps, HEIGHT, WIDTH, DEPTH])

    return inputs, labels

In [83]:
(0.1 + 0.2001)/1.2001

0.2500624947921007